In [12]:
import os
SNAPPY_notebook_path = os.path.join(os.path.abspath(""), "bench.ipynb")

time: 267 µs (started: 2024-05-07 16:56:49 -04:00)


In [13]:
import time
SNAPPY_start_time = time.perf_counter_ns()

time: 2.26 ms (started: 2024-05-07 16:56:49 -04:00)


In [14]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 474 µs (started: 2024-05-07 16:56:49 -04:00)


In [15]:
import numpy as np # linear algebra
import os
# STEFANOS: Conditionally import Modin Pandas
import pandas as pd




time: 221 µs (started: 2024-05-07 16:56:49 -04:00)


In [16]:
def hash_anything(obj):
    import pandas.util
    if isinstance(obj, pd.DataFrame):
        return pandas.util.hash_pandas_object(obj, index=False).to_numpy().data
    elif isinstance(obj, np.ndarray):
        return obj.data
    elif isinstance(obj, list):
        return str(obj).encode()
    else:
        return str(obj).encode()


def hash_dataframe(df):
    import xxhash
    h = xxhash.xxh64()
    for column in df.round(6).columns:
        h.update(hash_anything(df[column]))
    return h.digest()


time: 701 µs (started: 2024-05-07 16:56:49 -04:00)


In [17]:
# load & cleanup
file = os.path.abspath('') + '/input/indian-startup-recognized-by-dpiit/Startup_Counts_Across_India.csv'
df = pd.read_csv(file)

time: 3.77 ms (started: 2024-05-07 16:56:49 -04:00)


# -- STEFANOS -- Replicate Data

In [18]:
factor = 3000
df = pd.concat([df]*factor)
# df.info()

time: 359 ms (started: 2024-05-07 16:56:49 -04:00)


In [19]:
df.drop('S No.',axis=1,inplace=True)
df.dropna(inplace=True)
df.reset_index(inplace=True,drop=True)

#view
df.head()

,Year,State,Industry,Count
0,2022,Andaman and Nicobar Islands,Agriculture,1
1,2022,Andaman and Nicobar Islands,AR VR (Augmented + Virtual Reality),1
2,2022,Andaman and Nicobar Islands,Construction,1
3,2022,Andaman and Nicobar Islands,Internet of Things,1
4,2022,Andaman and Nicobar Islands,Marketing,1


time: 1.88 s (started: 2024-05-07 16:56:49 -04:00)


In [20]:
# %%time
# Industry sub-categories for environmental & AI startups
env = ['Agriculture','Green Technology','Renewable Energy','Waste Management']
ai = ['AI','Robotics','Computer Vision']

# combined df - environmental & AI startups only
df_ea = df.loc[(df['Industry'].isin(env)) | (df['Industry'].isin(ai))].reset_index(drop=True,inplace=False)

# custom function to set Main Industry
def set_MainIndustry(ind):
    if ind in env:
        return 'ENV'
    else:
        return 'AI'

# adding a new column
df_ea['MainIndustry'] = df_ea.Industry.apply(lambda x: set_MainIndustry(x))

# basic stats
print(f"A total of {df_ea.shape[0]} startups were started in India between 2016 & 2022, out of which {df_ea.groupby('MainIndustry').size()['ENV']} are environmental related & {df_ea.groupby('MainIndustry').size()['AI']} are AI startups.")

A total of 2361000 startups were started in India between 2016 & 2022, out of which 1473000 are environmental related & 888000 are AI startups.
time: 1.08 s (started: 2024-05-07 16:56:51 -04:00)


In [21]:
print(hash_dataframe(df_ea))

b'\xe5\xb5\xc4@\xd2\x03k\xd1'
time: 32.1 ms (started: 2024-05-07 16:56:52 -04:00)


In [22]:
SNAPPY_end_time = time.perf_counter_ns()
print("Total elapsed time:", (SNAPPY_end_time - SNAPPY_start_time) / (10 ** 9))

Total elapsed time: 3.40474768
time: 299 µs (started: 2024-05-07 16:56:52 -04:00)


# -- STEFANOS -- Disable the rest of the code because it's plotting